<a href="https://colab.research.google.com/github/marco-siino/GM_SOURCE_CODE/blob/main/NVIDIA_DS/CodeT5/GM_NVIDIA_DS_FOLD_3_CodeT5_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Setup Development Environment

Our first step is to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages.

## Import dependencies

In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
import os
import re
import glob
from datasets import load_dataset
import datasets
import torch
from sklearn.model_selection import KFold
from datasets import load_dataset
from datasets import Dataset
import random
from datasets.dataset_dict import DatasetDict
from datasets import concatenate_datasets

## 2. Load and prepare the dataset



In [2]:
# Import the csv into pandas dataframe and add the headers

df = pd.read_csv('dataset-devmap-nvidia.csv',sep=',')
#df = pd.concat(map(pd.read_csv, ['dataset-devmap-nvidia.csv', 'dataset-devmap-amd.csv']))
#df = pd.read_csv('dataset-devmap-nvidia.csv', sep='\t', names=['benchmark','dataset','comp','rational','mem','localmem','coalesced','atomic','transfer','wgsize','oracle','runtime_cpu','runtime_gpu','src','seq'])
#print(df.head())
# Now include transfer and wgsize columns into the src column.
df['src'] = df['transfer'].astype(str) +" - "+ df['wgsize'].astype(str) +" - "+df["src"]
# # Removing unwanted columns and only leaving title of news and the category which will be the target
df = df[['src','oracle']]
#print(df.head())

encode_dict = {}

def encode_cat(x):
    if x == "GPU":
      encode_dict[x]=1
    else:
      encode_dict[x]=0
    return encode_dict[x]

df['ENCODE_CAT'] = df['oracle'].apply(lambda x: encode_cat(x))

print(df)

df = df.sample(frac=1, random_state=1).reset_index(drop=True)

print(df)

                                                   src oracle  ENCODE_CAT
0    2048 - 255 - __kernel void A(int a, const __gl...    GPU           1
1    131072 - 256 - __kernel void A(__global uint* ...    GPU           1
2    3145728 - 256 - extern void B(float4 a, float4...    GPU           1
3    4096 - 256 - __kernel void A(__global float* a...    GPU           1
4    524288 - 256 - __kernel void A(__global uint* ...    CPU           0
..                                                 ...    ...         ...
675  2000628 - 128 - __kernel void A(__global const...    CPU           0
676  2000628 - 128 - __kernel void A(__global const...    CPU           0
677  71647488 - 0 - extern int D(__private int, __p...    CPU           0
678  71647488 - 256 - extern int B(int, int, int);\...    CPU           0
679  117440512 - 128 - __kernel void A(__global con...    CPU           0

[680 rows x 3 columns]
                                                   src oracle  ENCODE_CAT
0    6346800 -

In [3]:
df = df[['src','ENCODE_CAT']]
df= df.rename(columns={"src": "text", "ENCODE_CAT": "label"})
print(df)

                                                  text  label
0    6346800 - 52 - __kernel void A(__global double...      0
1    15949464 - 48 - __kernel void A(__global const...      1
2    14742140 - 128 - extern double __clc_pow(const...      0
3    15948384 - 2 - extern void B(double m, double ...      0
4    755867256 - 64 - __kernel void A(__global doub...      1
..                                                 ...    ...
675  14742140 - 128 - __kernel void A(__global doub...      1
676  57594120 - 128 - __kernel void A(__global cons...      0
677  320454824 - 64 - extern void B(double, double,...      1
678  1344811536 - 26 - extern void D(__private doub...      1
679  1313673512 - 32 - __kernel void A(__global con...      1

[680 rows x 2 columns]


In [4]:
# Each i-train fold can be accessed with df_train[i]. Same for test.
fold_nr = 5

kf = KFold(n_splits=5, random_state=None, shuffle=False)

df_train = []
df_test = []
model = []

for i, (train_index, test_index) in enumerate(kf.split(df)):
  df_train.append(df.iloc[train_index])
  df_test.append(df.iloc[test_index])
# print(df_train[0])

for i in range(0,fold_nr):
  df_train[i] = df_train[i].reset_index(drop=True)
  df_test[i] = df_test[i].reset_index(drop=True)

In [5]:
fold = 3

dataset = {'train':Dataset.from_dict({'text': df_train[fold]['text'],'label':df_train[fold]['label']}),
     'test':Dataset.from_dict({'text': df_test[fold]['text'],'label':df_test[fold]['label']})
     }
dataset = DatasetDict({'train': dataset['train'], 'test': dataset['test']})
DatasetDict(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 544
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 136
    })
})

Lets checkout an example of the dataset.

In [6]:
df_train[4]['text']

0      6346800 - 52 - __kernel void A(__global double...
1      15949464 - 48 - __kernel void A(__global const...
2      14742140 - 128 - extern double __clc_pow(const...
3      15948384 - 2 - extern void B(double m, double ...
4      755867256 - 64 - __kernel void A(__global doub...
                             ...                        
539    1313673032 - 32 - __kernel void A(__global dou...
540    804152 - 60 - __kernel void A(__global double*...
541    98304 - 64 - extern double B(int, double *, do...
542    57594120 - 128 - __kernel void A(__global cons...
543    6291464 - 64 - extern void B(float2 *, float2 ...
Name: text, Length: 544, dtype: object

In [ ]:
dataset['train'][1]

{'text': '15949464 - 48 - __kernel void A(__global const double* a, __global double* b, __global double* c, __global double* d, __global double* e, __global double* f, __global double* g, int h, int i, int j) {\n  int k, l, m;\n  double n;\n  double o[4];\n  double p;\n\n  m = get_global_id(2);\n  l = get_global_id(1);\n  k = get_global_id(0);\n  if (m >= j || l >= i || k >= h)\n    return;\n\n  __global double(*q)[12 / 2 * 2 + 1][12 / 2 * 2 + 1][5] = (__global double(*)[12 / 2 * 2 + 1][12 / 2 * 2 + 1][5])a;\n  __global double(*r)[12 / 2 * 2 + 1][12 / 2 * 2 + 1] = (__global double(*)[12 / 2 * 2 + 1][12 / 2 * 2 + 1])b;\n  __global double(*s)[12 / 2 * 2 + 1][12 / 2 * 2 + 1] = (__global double(*)[12 / 2 * 2 + 1][12 / 2 * 2 + 1])c;\n  __global double(*t)[12 / 2 * 2 + 1][12 / 2 * 2 + 1] = (__global double(*)[12 / 2 * 2 + 1][12 / 2 * 2 + 1])d;\n  __global double(*u)[12 / 2 * 2 + 1][12 / 2 * 2 + 1] = (__global double(*)[12 / 2 * 2 + 1][12 / 2 * 2 + 1])e;\n  __global double(*v)[12 / 2 * 2 + 1]

# To train our model we need to convert our inputs (text) to token IDs. This is done by a 🤗 Transformers Tokenizer. If you are not sure what this means check out [chapter 6](https://huggingface.co/course/chapter6/1?fw=tf) of the Hugging Face Course.

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_id="Salesforce/codet5-base"
#model_id = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)

before we can start training we need to preprocess our data. Abstractive flan t5 is good for a text2text-generation task. This means our model will take a text as input and generate a text as output. For this we want to understand how long our input and output will be to be able to efficiently batch our data.
- as result, we should to convert label from int to string

In [8]:
# dataset['train'] = dataset['train'].shuffle(seed=42).select(range(2000))
# dataset['test'] = dataset['test'].shuffle(seed=42).select(range(1000))

#dataset['train'] = dataset['train'].shuffle(seed=42)

# per ora faccio solo fold 0.
#train_df = pd.DataFrame(dataset['train'])
#test_df = pd.DataFrame(dataset['test'])
train_df = df_train[fold]
test_df = df_test[fold]


dataset.clear()
train_df['label'] = train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)
dataset['train'] = Dataset.from_pandas(train_df)
dataset['test'] = Dataset.from_pandas(test_df)

In [9]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Max target length: 3


In [10]:
def preprocess_function(sample, padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    #tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/544 [00:00<?, ? examples/s]

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


## 3. Fine-tune and evaluate CodeT5




In [11]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="Salesforce/codet5-base"
#model_id = "codellama/CodeLlama-7b-hf"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)
#model = LlamaModel.from_pretrained("codellama/CodeLlama-7b-hf")

We want to evaluate our model during training. The `Trainer` supports evaluation during training by providing a `compute_metrics`.  
The most commonly used metrics to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries

We are going to use `evaluate` library to evaluate the `rogue` score.

In [12]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
#nltk.download("punkt")

# Metric
#metric = evaluate.load("f1")
metric = evaluate.load("evaluate/metrics/accuracy")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    #result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

Before we can start training is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library.

In [13]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training. We are leveraging the [Hugging Face Hub](https://huggingface.co/models) integration of the `Trainer` to automatically push our checkpoints, logs and metrics during training into a repository.

In [14]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-imdb-text-classification"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-4,

    num_train_epochs=10,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="epoch",
    # logging_steps=1000,
    evaluation_strategy="no",
    save_strategy = "no",
    #save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


We can start our training by using the `train` method of the `Trainer`.

In [15]:
# Start training
trainer.train()

/leonardo/home/userexternal/msiino00/clean-updated-env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
34,0.931000
68,0.254800
102,0.234500
136,0.227000
170,0.216000
204,0.208100
238,0.203500
272,0.172000
306,0.154200
340,0.145100


TrainOutput(global_step=340, training_loss=0.27461793562945197, metrics={'train_runtime': 156.4662, 'train_samples_per_second': 34.768, 'train_steps_per_second': 2.173, 'total_flos': 3312730924646400.0, 'train_loss': 0.27461793562945197, 'epoch': 10.0})

Nice, we have trained our model. 🎉 Lets run evaluate the best model again on the test set.


In [16]:
trainer.evaluate()

/leonardo/home/userexternal/msiino00/clean-updated-env/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 0.21799100935459137,
 'eval_accuracy': 72.7941,
 'eval_gen_len': 3.0,
 'eval_runtime': 5.1286,
 'eval_samples_per_second': 26.518,
 'eval_steps_per_second': 1.755,
 'epoch': 10.0}

In [ ]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

## 4. Run Inference and Classification Report

In [ ]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = dataset['test']['text'][i]
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['label'][i])

  progress_bar.update(1)

In [ ]:
str_labels_list = []
for i in range(len(labels_list)): str_labels_list.append(str(labels_list[i]))

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list, zero_division=0)
print(report)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list, zero_division=0)
print(report)